In [ ]:
! pip install git+https://github.com/unifyai/models.git

# layers

In [10]:
from typing import Callable, Optional, Union, Tuple, List
import ivy


class InceptionBasicConv2d(ivy.Module):
    """
    InceptionBasicConv2d used in the ResNet architecture.

    Args::
        inplanes (int): Number of input channels.
        planes (int): Number of output channels.
        stride (int): Stride value for the block. Defaults to 1.
        kernel_size (int): size of kernel.

    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: Union[List, Tuple],
        stride: int = 1,
        padding: int = 0,
    ) -> None:
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        super(InceptionBasicConv2d, self).__init__()

    def _build(self, *args, **kwargs):
        self.conv = ivy.Conv2D(
            self.in_channels,
            self.out_channels,
            self.kernel_size,
            self.stride,
            self.padding,
            with_bias=False,
        )
        self.bn = ivy.BatchNorm2D(self.out_channels, eps=0.001)

    def _forward(self, x):
        """Forward pass method for the module."""
        x = self.conv(x)
        x = self.bn(x)
        x = ivy.relu(x)
        return x


class InceptionAux(ivy.Module):
    """
    InceptionAux used in the ResNet architecture.

    Args::
        inplanes (int): Number of input channels.
        conv_block (int): InceptionBasicConv2d module

    """

    def __init__(
        self,
        in_channels: int,
        num_classes: int,
        conv_block: Optional[Callable[..., ivy.Module]] = None,
    ) -> None:
        if conv_block is None:
            self.conv_block = InceptionBasicConv2d
        self.in_channels = in_channels
        self.num_classes = num_classes
        super().__init__()

    def _build(self, *args, **kwargs):
        self.conv0 = self.conv_block(self.in_channels, 128, kernel_size=[1, 1])
        self.conv1 = self.conv_block(128, 768, kernel_size=[5, 5])
        self.conv1.stddev = 0.01
        self.fc = ivy.Linear(768, self.num_classes, with_bias=False)
        self.fc.stddev = 0.001

    def _forward(self, x):
        # N x 768 x 17 x 17
        x = ivy.avg_pool2d(x, (5, 5), (3, 3), "valid")

        # N x 768 x 5 x 5
        x = self.conv0(x)

        # N x 128 x 5 x 5
        x = self.conv1(x)

        # N x 768 x 1 x 1
        # Adaptive average pooling
        x = ivy.permute_dims(x, (0, 3, 1, 2))
        x = ivy.adaptive_avg_pool2d(x, (1, 1))
        x = ivy.permute_dims(x, (0, 2, 3, 1))

        # N x 768 x 1 x 1
        x = ivy.flatten(x, start_dim=1)

        # N x 768
        x = self.fc(x)
        # N x 1000
        return x


class InceptionA(ivy.Module):
    """
    InceptionA used in the ResNet architecture.

    Args::
        inplanes (int): Number of input channels.
        conv_block (int): InceptionBasicConv2d module

    """

    def __init__(
        self,
        in_channels: int,
        pool_features: int,
        conv_block: Optional[Callable[..., ivy.Module]] = None,
    ) -> None:
        if conv_block is None:
            self.conv_block = InceptionBasicConv2d
        self.in_channels = in_channels
        self.pool_features = pool_features
        super().__init__()

    def _build(self, *args, **kwargs):
        self.branch1x1 = self.conv_block(self.in_channels, 64, kernel_size=[1, 1])

        self.branch5x5_1 = self.conv_block(self.in_channels, 48, kernel_size=[1, 1])
        self.branch5x5_2 = self.conv_block(
            48, 64, kernel_size=[5, 5], padding=[[2, 2], [2, 2]]
        )

        self.branch3x3dbl_1 = self.conv_block(self.in_channels, 64, kernel_size=[1, 1])
        self.branch3x3dbl_2 = self.conv_block(
            64, 96, kernel_size=[3, 3], padding=[[1, 1], [1, 1]]
        )
        self.branch3x3dbl_3 = self.conv_block(
            96, 96, kernel_size=[3, 3], padding=[[1, 1], [1, 1]]
        )

        self.branch_pool = self.conv_block(
            self.in_channels, self.pool_features, kernel_size=[1, 1]
        )

    def _forward(self, x: ivy.Array) -> List[ivy.Array]:
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = ivy.avg_pool2d(x, (3, 3), (1, 1), ((1, 1), (1, 1)))

        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        outputs = ivy.concat(outputs, axis=3)

        return outputs


class InceptionB(ivy.Module):
    """
    InceptionB used in the ResNet architecture.

    Args::
        inplanes (int): Number of input channels.
        conv_block (int): InceptionBasicConv2d module

    """

    def __init__(
        self, in_channels: int, conv_block: Optional[Callable[..., ivy.Module]] = None
    ) -> None:
        if conv_block is None:
            self.conv_block = InceptionBasicConv2d
        self.in_channels = in_channels
        super().__init__()

    def _build(self, *args, **kwargs):
        self.branch3x3 = self.conv_block(
            self.in_channels, 384, kernel_size=[3, 3], stride=2
        )

        self.branch3x3dbl_1 = self.conv_block(self.in_channels, 64, kernel_size=[1, 1])
        self.branch3x3dbl_2 = self.conv_block(
            64, 96, kernel_size=[3, 3], padding=[[1, 1], [1, 1]]
        )
        self.branch3x3dbl_3 = self.conv_block(96, 96, kernel_size=[3, 3], stride=2)

    def _forward(self, x: ivy.Array) -> List[ivy.Array]:
        branch3x3 = self.branch3x3(x)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = ivy.max_pool2d(x, [3, 3], 2, 0)

        outputs = [branch3x3, branch3x3dbl, branch_pool]
        outputs = ivy.concat(outputs, axis=3)

        return outputs


class InceptionC(ivy.Module):
    """
    InceptionC used in the ResNet architecture.

    Args::
        inplanes (int): Number of input channels.
        conv_block (int): InceptionBasicConv2d module

    """

    def __init__(
        self,
        in_channels: int,
        channels_7x7: int,
        conv_block: Optional[Callable[..., ivy.Module]] = None,
    ) -> None:
        if conv_block is None:
            self.conv_block = InceptionBasicConv2d
        self.in_channels = in_channels
        self.channels_7x7 = channels_7x7
        super().__init__()

    def _build(self, *args, **kwargs):
        self.branch1x1 = self.conv_block(self.in_channels, 192, kernel_size=[1, 1])

        c7 = self.channels_7x7
        self.branch7x7_1 = self.conv_block(self.in_channels, c7, kernel_size=[1, 1])
        self.branch7x7_2 = self.conv_block(
            c7, c7, kernel_size=[1, 7], padding=[[0, 0], [3, 3]]
        )
        self.branch7x7_3 = self.conv_block(
            c7, 192, kernel_size=[7, 1], padding=[[3, 3], [0, 0]]
        )
        self.branch7x7dbl_1 = self.conv_block(self.in_channels, c7, kernel_size=[1, 1])
        self.branch7x7dbl_2 = self.conv_block(
            c7, c7, kernel_size=[7, 1], padding=[[3, 3], [0, 0]]
        )
        self.branch7x7dbl_3 = self.conv_block(
            c7, c7, kernel_size=[1, 7], padding=[[0, 0], [3, 3]]
        )
        self.branch7x7dbl_4 = self.conv_block(
            c7, c7, kernel_size=[7, 1], padding=[[3, 3], [0, 0]]
        )
        self.branch7x7dbl_5 = self.conv_block(
            c7, 192, kernel_size=[1, 7], padding=[[0, 0], [3, 3]]
        )

        self.branch_pool = self.conv_block(self.in_channels, 192, kernel_size=[1, 1])

    def _forward(self, x: ivy.Array) -> List[ivy.Array]:
        branch1x1 = self.branch1x1(x)

        branch7x7 = self.branch7x7_1(x)
        branch7x7 = self.branch7x7_2(branch7x7)
        branch7x7 = self.branch7x7_3(branch7x7)

        branch7x7dbl = self.branch7x7dbl_1(x)
        branch7x7dbl = self.branch7x7dbl_2(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_3(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_4(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_5(branch7x7dbl)

        branch_pool = ivy.avg_pool2d(x, (3, 3), (1, 1), ((1, 1), (1, 1)))
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch7x7, branch7x7dbl, branch_pool]
        outputs = ivy.concat(outputs, axis=3)

        return outputs


class InceptionD(ivy.Module):
    """
    InceptionD used in the ResNet architecture.

    Args::
        inplanes (int): Number of input channels.
        conv_block (int): InceptionBasicConv2d module

    """

    def __init__(
        self, in_channels: int, conv_block: Optional[Callable[..., ivy.Module]] = None
    ) -> None:
        if conv_block is None:
            self.conv_block = InceptionBasicConv2d
        self.in_channels = in_channels
        super().__init__()

    def _build(self, *args, **kwargs):
        self.branch3x3_1 = self.conv_block(self.in_channels, 192, kernel_size=[1, 1])
        self.branch3x3_2 = self.conv_block(192, 320, kernel_size=[3, 3], stride=2)

        self.branch7x7x3_1 = self.conv_block(self.in_channels, 192, kernel_size=[1, 1])
        self.branch7x7x3_2 = self.conv_block(
            192, 192, kernel_size=[1, 7], padding=[[0, 0], [3, 3]]
        )
        self.branch7x7x3_3 = self.conv_block(
            192, 192, kernel_size=[7, 1], padding=[[3, 3], [0, 0]]
        )
        self.branch7x7x3_4 = self.conv_block(192, 192, kernel_size=[3, 3], stride=2)

    def _forward(self, x: ivy.Array) -> List[ivy.Array]:
        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)

        branch7x7x3 = self.branch7x7x3_1(x)
        branch7x7x3 = self.branch7x7x3_2(branch7x7x3)
        branch7x7x3 = self.branch7x7x3_3(branch7x7x3)
        branch7x7x3 = self.branch7x7x3_4(branch7x7x3)

        branch_pool = ivy.max_pool2d(x, [3, 3], 2, 0)

        outputs = [branch3x3, branch7x7x3, branch_pool]
        outputs = ivy.concat(outputs, axis=3)

        return outputs


class InceptionE(ivy.Module):
    """
    InceptionE used in the ResNet architecture.

    Args::
        inplanes (int): Number of input channels.
        conv_block (int): InceptionBasicConv2d module

    """

    def __init__(
        self, in_channels: int, conv_block: Optional[Callable[..., ivy.Module]] = None
    ) -> None:
        if conv_block is None:
            self.conv_block = InceptionBasicConv2d
        self.in_channels = in_channels
        super().__init__()

    def _build(self, *args, **kwargs):
        self.branch1x1 = self.conv_block(self.in_channels, 320, kernel_size=[1, 1])

        self.branch3x3_1 = self.conv_block(self.in_channels, 384, kernel_size=[1, 1])
        self.branch3x3_2a = self.conv_block(
            384, 384, kernel_size=[1, 3], padding=[[0, 0], [1, 1]]
        )
        self.branch3x3_2b = self.conv_block(
            384, 384, kernel_size=[3, 1], padding=[[1, 1], [0, 0]]
        )

        self.branch3x3dbl_1 = self.conv_block(self.in_channels, 448, kernel_size=[1, 1])
        self.branch3x3dbl_2 = self.conv_block(448, 384, kernel_size=[3, 3], padding=1)
        self.branch3x3dbl_3a = self.conv_block(
            384, 384, kernel_size=[1, 3], padding=[[0, 0], [1, 1]]
        )
        self.branch3x3dbl_3b = self.conv_block(
            384, 384, kernel_size=[3, 1], padding=[[1, 1], [0, 0]]
        )

        self.branch_pool = self.conv_block(self.in_channels, 192, kernel_size=[1, 1])

    def _forward(self, x: ivy.Array) -> List[ivy.Array]:
        branch1x1 = self.branch1x1(x)

        branch3x3 = self.branch3x3_1(x)
        branch3x3 = ivy.concat(
            [
                self.branch3x3_2a(branch3x3),
                self.branch3x3_2b(branch3x3),
            ],
            axis=3,
        )

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = ivy.concat(
            [
                self.branch3x3dbl_3a(branch3x3dbl),
                self.branch3x3dbl_3b(branch3x3dbl),
            ],
            axis=3,
        )

        branch_pool = ivy.avg_pool2d(x, (3, 3), (1, 1), [(1, 1), (1, 1)])
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch3x3, branch3x3dbl, branch_pool]
        outputs = ivy.concat(outputs, axis=3)
        return outputs


# weights prep

In [11]:
import ivy_models
import torch

In [12]:

def _prune_keys(raw, ref, raw_keys_to_prune=[], ref_keys_to_prune=[]):
    pruned_ref = {}
    if raw_keys_to_prune:
        raw = raw.cont_prune_keys(raw_keys_to_prune)
    if ref_keys_to_prune:
        pruned_ref = ref.cont_at_keys(ref_keys_to_prune)
        ref = ref.cont_prune_keys(ref_keys_to_prune)
    return raw, ref, pruned_ref


def _map_weights(raw, ref, custom_mapping=None):
    mapping = {}
    for old_key, new_key in zip(
        raw.cont_sort_by_key().cont_to_iterator_keys(),
        ref.cont_sort_by_key().cont_to_iterator_keys(),
    ):
        new_mapping = new_key
        if custom_mapping is not None:
            new_mapping = custom_mapping(old_key, new_key)
        mapping[old_key] = new_mapping
    return mapping


def load_torch_weights(
    url,
    ref_model,
    raw_keys_to_prune=[],
    ref_keys_to_prune=[],
    custom_mapping=None,
    map_location=torch.device("cpu"),
):
    ivy_torch = ivy.with_backend("torch")
    weights = torch.hub.load_state_dict_from_url(url, map_location=map_location)


    weights_raw = ivy.Container(
        ivy_torch.to_numpy(ivy_torch.Container(weights)).cont_to_dict()
    )
    weights_raw, weights_ref, pruned_ref = _prune_keys(
        weights_raw, ref_model.v, raw_keys_to_prune, ref_keys_to_prune
    )
    mapping = _map_weights(weights_raw, weights_ref, custom_mapping=custom_mapping)
    w_clean = weights_raw.cont_restructure(mapping, keep_orig=False)
    if ref_keys_to_prune:
        w_clean = ivy.Container.cont_combine(w_clean, pruned_ref)
    
    display(f"2 torch model weights are:")
    display(ivy.asarray(ivy.Container(weights)))

    return ivy.asarray(w_clean)



# inception

In [14]:
# inspired heavily by pytorch's efficient -
# https://github.com/pytorch/vision/blob/main/torchvision/models/inception.py

import ivy
import ivy_models
# from ivy_models.inceptionnet.layers import (
#     InceptionBasicConv2d,
#     InceptionAux,
#     InceptionA,
#     InceptionB,
#     InceptionC,
#     InceptionD,
#     InceptionE,
# )
from ivy_models.base import BaseSpec, BaseModel
from typing import List


class InceptionNetSpec(BaseSpec):
    def __init__(
        self, num_classes=1000, training=False, dropout=0.5, data_format="NHWC"
    ):
        super(InceptionNetSpec, self).__init__(
            training=training,
            num_classes=num_classes,
            dropout=dropout,
            data_format=data_format,
        )


class InceptionV3(BaseModel):
    """An Ivy native implementation of InceptionNet"""

    """
    Args:
        ----
            num_classes (int): Number of classes
            training (bool): Set the mode for model execution
                # If training=True, the model will be in training mode
                # If training=False, the model will be in evaluation mode
            dropout (float): The droupout probability
    """

    def __init__(
        self,
        training: bool = False,
        num_classes: int = 1000,
        dropout: float = 0.5,
        data_format: str = "NHWC",
        spec=None,
        v=None,
    ) -> None:
        if not training:
            dropout = 0
        self.spec = (
            spec
            if spec and isinstance(spec, InceptionNetSpec)
            else InceptionNetSpec(
                training=training,
                num_classes=num_classes,
                dropout=dropout,
                data_format=data_format,
            )
        )
        super(InceptionV3, self).__init__(v=v)

    def _build(self, *args, **kwargs):
        conv_block = InceptionBasicConv2d
        inception_a = InceptionA
        inception_b = InceptionB
        inception_c = InceptionC
        inception_d = InceptionD
        inception_e = InceptionE
        inception_aux = InceptionAux

        self.Conv2d_1a_3x3 = conv_block(3, 32, kernel_size=[3, 3], stride=2)
        self.Conv2d_2a_3x3 = conv_block(32, 32, kernel_size=[3, 3])
        self.Conv2d_2b_3x3 = conv_block(
            32, 64, kernel_size=[3, 3], padding=[[1, 1], [1, 1]]
        )
        self.maxpool1 = ivy.MaxPool2D([3, 3], 2, 0)
        self.Conv2d_3b_1x1 = conv_block(64, 80, kernel_size=[1, 1])
        self.Conv2d_4a_3x3 = conv_block(80, 192, kernel_size=[3, 3])
        self.maxpool2 = ivy.MaxPool2D([3, 3], 2, 0)
        self.Mixed_5b = inception_a(192, pool_features=32)
        self.Mixed_5c = inception_a(256, pool_features=64)
        self.Mixed_5d = inception_a(288, pool_features=64)
        self.Mixed_6a = inception_b(288)
        self.Mixed_6b = inception_c(768, channels_7x7=128)
        self.Mixed_6c = inception_c(768, channels_7x7=160)
        self.Mixed_6d = inception_c(768, channels_7x7=160)
        self.Mixed_6e = inception_c(768, channels_7x7=192)

        # if is used only when the model is in training mode
        if self.spec.training:
            self.AuxLogits = inception_aux(768, self.spec.num_classes)

        self.Mixed_7a = inception_d(768)
        self.Mixed_7b = inception_e(1280)
        self.Mixed_7c = inception_e(2048)
        self.avgpool = ivy.AdaptiveAvgPool2d((1, 1))
        self.dropout = ivy.Dropout(prob=self.spec.dropout, training=self.spec.training)
        self.fc = ivy.Linear(2048, self.spec.num_classes, with_bias=True)

    @classmethod
    def get_spec_class(self):
        return InceptionNetSpec

    def _forward(self, x: ivy.Array, data_format=None) -> List[ivy.Array]:
        data_format = data_format if data_format else self.spec.data_format
        if data_format == "NCHW":
            x = ivy.permute_dims(x, (0, 2, 3, 1))

        # N x 3 x 299 x 299
        x = self.Conv2d_1a_3x3(x)
        # N x 32 x 149 x 149

        x = self.Conv2d_2a_3x3(x)
        # N x 32 x 147 x 147

        x = self.Conv2d_2b_3x3(x)
        # N x 64 x 147 x 147

        x = self.maxpool1(x)
        # N x 64 x 73 x 73

        x = self.Conv2d_3b_1x1(x)
        # N x 80 x 73 x 73

        x = self.Conv2d_4a_3x3(x)
        # N x 192 x 71 x 71

        x = self.maxpool2(x)
        # N x 192 x 35 x 35

        x = self.Mixed_5b(x)
        # N x 256 x 35 x 35

        x = self.Mixed_5c(x)
        # N x 288 x 35 x 35

        x = self.Mixed_5d(x)
        # N x 288 x 35 x 35

        x = self.Mixed_6a(x)
        # N x 768 x 17 x 17

        x = self.Mixed_6b(x)
        # N x 768 x 17 x 17

        x = self.Mixed_6c(x)
        # N x 768 x 17 x 17

        x = self.Mixed_6d(x)
        # N x 768 x 17 x 17

        x = self.Mixed_6e(x)
        # N x 768 x 17 x 17

        aux = None
        if self.spec.training:
            aux = self.AuxLogits(x)
            # N x 768 x 17 x 17

        x = self.Mixed_7a(x)
        # N x 1280 x 8 x 8

        x = self.Mixed_7b(x)
        # N x 2048 x 8 x 8

        x = self.Mixed_7c(x)
        # N x 2048 x 8 x 8

        # Adaptive average pooling
        x = ivy.permute_dims(x, (0, 3, 1, 2))
        x = self.avgpool(x)
        x = ivy.permute_dims(x, (0, 2, 3, 1))

        # N x 2048 x 1 x 1

        x = self.dropout(x)
        # N x 2048 x 1 x 1

        x = ivy.flatten(x, start_dim=1)
        # N x 2048

        x = self.fc(x)
        # N x 1000 (num_classes)

        return x, aux


def _inceptionNet_v3_torch_weights_mapping(old_key, new_key):
    if "conv/weight" in old_key:
        return {"key_chain": new_key, "pattern": "b c h w -> h w c b"}
    return new_key


def inceptionNet_v3(
    pretrained=True, training=False, num_classes=1000, dropout=0.5, data_format="NHWC"
):
    """InceptionNet-V3 model"""
    model = InceptionV3(
        training=training,
        num_classes=num_classes,
        dropout=dropout,
        data_format=data_format,
    )
    display(f"1 my model weights are:")
    display(model.v)
    if pretrained:
        url = "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth"
        w_clean = load_torch_weights(
            url,
            model,
            raw_keys_to_prune=["num_batches_tracked", "AuxLogits"],
            custom_mapping=_inceptionNet_v3_torch_weights_mapping,
        )

        model.v = w_clean
    return model


# test

In [15]:
ivy.set_backend('torch')

import os
from ivy_models_tests import helpers

# Load image
img = helpers.load_and_preprocess_img(
    os.path.join("/workspaces/models_Ivy_Sark42/images/dog.jpg"),
    256,
    224,
    data_format="NHWC",
    to_ivy=True,
)


def test_final_block():
    random_test_tensor = ivy.random_normal(shape=(1, 224, 224, 3))
    display(f"random_test_tensor shape is: {random_test_tensor.shape}")
    
    conv_block = inceptionNet_v3()
    logits, _ = conv_block(img)
    display(f"logits is: {logits}")

    # Cardinality test
    assert logits.shape == tuple([ivy.to_scalar(1), 1000])

    import numpy as np
    # Value test
    if True:
        np_out = ivy.to_numpy(logits[0])
        true_indices = np.array([258, 270, 279])
        calc_indices = np.argsort(np_out)[-3:][::-1]
        display(f"calc_indices is: {calc_indices}")
        assert np.array_equal(true_indices, calc_indices)

        true_logits = np.array([9.9990e-01, 8.3909e-05, 1.1693e-05])
        calc_logits = np.take(np_out, calc_indices)
        assert np.allclose(true_logits, calc_logits, rtol=1)
test_final_block()

'random_test_tensor shape is: ivy.Shape(1, 224, 224, 3)'

'1 my model weights are:'

{
    Conv2d_1a_3x3: {
        bn: {
            b: (<class ivy.data_classes.array.array.Array> shape=[32]),
            running_mean: (<class ivy.data_classes.array.array.Array> shape=[32]),
            running_var: (<class ivy.data_classes.array.array.Array> shape=[32]),
            w: (<class ivy.data_classes.array.array.Array> shape=[32])
        },
        conv: {
            w: (<class ivy.data_classes.array.array.Array> shape=[3, 3, 3, 32])
        }
    },
    Conv2d_2a_3x3: {
        bn: {
            b: (<class ivy.data_classes.array.array.Array> shape=[32]),
            running_mean: (<class ivy.data_classes.array.array.Array> shape=[32]),
            running_var: (<class ivy.data_classes.array.array.Array> shape=[32]),
            w: (<class ivy.data_classes.array.array.Array> shape=[32])
        },
        conv: {
            w: (<class ivy.data_classes.array.array.Array> shape=[3, 3, 32, 32])
        }
    },
    Conv2d_2b_3x3: {
        bn: {
            b: (<class ivy.d

'2 torch model weights are:'

{
    AuxLogits: {
        conv0: {
            bn: {
                bias: (<class ivy.data_classes.array.array.Array> shape=[128]),
                running_mean: (<class ivy.data_classes.array.array.Array> shape=[128]),
                running_var: (<class ivy.data_classes.array.array.Array> shape=[128]),
                weight: (<class ivy.data_classes.array.array.Array> shape=[128])
            },
            conv: {
                weight: (<class ivy.data_classes.array.array.Array> shape=[128, 768, 1, 1])
            }
        },
        conv1: {
            bn: {
                bias: (<class ivy.data_classes.array.array.Array> shape=[768]),
                running_mean: (<class ivy.data_classes.array.array.Array> shape=[768]),
                running_var: (<class ivy.data_classes.array.array.Array> shape=[768]),
                weight: (<class ivy.data_classes.array.array.Array> shape=[768])
            },
            conv: {
                weight: (<class ivy.data_classes.arr

'logits is: ivy.array([[ 2.24186480e-02,  8.61005485e-02,  3.98007333e-02,\n        -1.02385864e-01, -5.92550039e-02, -7.44246915e-02,\n        -1.77067325e-01, -1.51681244e-01, -1.02749288e-01,\n        -2.28919893e-01, -1.57801434e-01, -9.65706557e-02,\n        -8.99402499e-02, -9.93840545e-02, -1.18451424e-01,\n        -8.23233873e-02, -3.17190960e-02, -9.57658887e-02,\n         4.19645011e-02, -3.79585624e-02, -1.15260392e-01,\n        -1.90309584e-01, -2.05121785e-01, -1.62723571e-01,\n        -2.61412561e-01, -2.44996995e-02,  2.98291445e-03,\n        -1.05413675e-01, -7.65306652e-02, -1.20341539e-01,\n        -1.63992137e-01, -3.89248878e-02,  3.83493863e-02,\n        -1.32076949e-01, -1.63869277e-01, -1.29390121e-01,\n         6.15689158e-02, -9.46835428e-02, -2.27183402e-02,\n         3.78623903e-02, -7.63765127e-02, -8.92683864e-02,\n        -6.79808557e-02, -1.28836632e-02, -8.81138444e-03,\n        -2.05534249e-02, -1.14935428e-01, -4.11531329e-02,\n        -8.96753967e-02,

'calc_indices is: [823 851 523]'

AssertionError: 